In [3]:
# Import necessary libraries
import json
import openai

# Load and prepare your dataset
with open('../data/chat_dataset.json', 'r') as f:
    data = json.load(f)

# Prepare the data in JSONL format
with open('../data/training_data.jsonl', 'w') as outfile:
    for entry in data:
        for message in entry['messages']:
            if message['role'] == 'user':
                prompt = message['content']
            elif message['role'] == 'assistant':
                completion = json.dumps(message['content'])
                jsonl_entry = {"prompt": prompt, "completion": completion}
                outfile.write(json.dumps(jsonl_entry) + "\n")

# Set up your OpenAI API key
openai.api_key = 'your-api-key'

# Upload the training file
response = openai.File.create(
    file=open("../data/training_data.jsonl"),
    purpose='fine-tune'
)
training_file_id = response['id']
print(f"Training file ID: {training_file_id}")

# Fine-tune the model
fine_tune_response = openai.FineTune.create(
    training_file=training_file_id,
    model="curie"  # or "davinci"
)
fine_tune_id = fine_tune_response['id']
print(f"Fine-tuning job ID: {fine_tune_id}")

# Check the status of the fine-tuning job
status = openai.FineTune.retrieve(fine_tune_id)
print(status)

# Use the fine-tuned model
response = openai.Completion.create(
    model=fine_tune_id,  # Use the ID of the fine-tuned model
    prompt="Provide a lesson plan on Introduction to Addition and Subtraction suitable for students with Dyslexia and ADHD, for Kindergarten through Grade 2.",
    max_tokens=1000
)

print(response['choices'][0]['text'])


FileNotFoundError: [Errno 2] No such file or directory: '../data/chat_dataset.json'